### Reguła 3 sigm

In [1]:
import numpy as np
w = [1, 2, 4, 6, 7, 8, 11, 12, 14, 35, 79]
mean = np.mean(np.array(w), axis=0)
sd = np.std(np.array(w), axis=0)
[x for x in w if (x > mean - 3 * sd) and (x < mean + 3 * sd)]

# Chyba chodzi o to, ze jak sie pojawią tu wszystkie te same wartosci ze zbioru, to znacze ze jest 0 wartosci odstających

[1, 2, 4, 6, 7, 8, 11, 12, 14, 35, 79]

### Reguła kwantylowa

In [ ]:
# do odpalenia w R

w = c(1, 2, 4, 6, 7, 8, 11, 12, 14, 35, 79) 
IQR = IQR(w)
start = quantile(w, 0.25) - 1.5 *IQR
stop = quantile(w, 0.75) +1.5 * IQR

a = w <start
b = w>stop


print(sum(a+b))

### ZADANIE 7 Z PREZENTACJI O MODELU RETENCJI (do odpalenia w R)
Za pomocą metody Kaplana-Meiera oblicz i podaj liczbę klientów, którzy odeszli, medianę czasu odejścia oraz łączną wartość CLV. Zinterpretuj medianę czasu odejścia. Przyjmij, że stopa dyskontowa wynosi 5%, a wysokość wpływów od klienta jest równa $25, przy czym płatność jest dokonywana na początku miesiąca („ z góry”). Wyniki zaokrąglij do części całkowitej.
bigT – zmienna czasowa (miesiące)
cancel – status klienta (1-odejście, 0-ocenzurowanie)
count – liczebność klientów

!!! Treść z prezentacji (dane z kodu pasują do resci z prezentacji): !!!

Oszacuj funkcję przeżycia, gęstości prawdopodobieństwa i hazardu w SAS.
Skorzystaj ze zbioru service5yr, zawierającego dane firmy świadczącej usługi z
okresu 5 lat. W zadaniu załóż m = $23.20 (płatność na początku okresu z góry)
oraz d = 1%. Oszacowania porównaj z wynikami uzyskanymi w modelu
prostym.


In [ ]:
library(ggplot2) # popular data visualisation package
library(dplyr) # popular package for data manipulation
library(broom) # here: used for extracting data from model summary
library(survival) # basic package for survival analysis
library(survminer) # package for summarizing and visualizing the results of survival analysis
library(ggfortify)

# importing dataset
service5yr <- read.csv2(file = "SERVICE5YR.csv", sep = ";", header = T)

head(service5yr, 10)

# Estimation with with Kaplan-Meier method*/
model <- survival::survfit(Surv(bigT, cancel) ~ 1,
                           data = service5yr, 
                           weights = count)
# model summary
model 
summary(model) # n to liczba zdarzen, events to liczba klientow co odeszli, mediana to mediana a clv sum(model_data$clv)

survminer::ggsurvplot(model,
           conf.int = TRUE, # show confidence intervals
           surv.median.line = "hv", # how to show lifetime median (here: horizontal & vertical lines)
           ggtheme = theme_bw())

sums = service5yr %>% summarise(sum_cancel = sum(cancel*count), # suma ludzi co odeszli
                       sum_bigT = sum(bigT*count)) #liczba szans na odejście 

sums # stąd odczytać wartości i podmienić do wzoru rhat niżej 
        
# Comparing estimates from SRM and GRM

model_data <- data_frame(bigT = model$time, survival = model$surv)
model_data <- model_data %>% 
  dplyr::mutate(rhat = 1 - 44367/ 1073935, # w liczniku "sum_cancel" a w mianowniku zmienić na "sum_bigT" z "sums" co byly wyliczone wczesniej
                clv = survival*23.20/(1.01^(bigT-1))) %>% #tutaj zmienić 23.30na inna kwote i  d=1 na d=(wartosc w poleceniu) czyli zamiast 1.01 to 1.05
  dplyr::mutate(survSRM = (rhat)^(bigT-1),
                clvSRM = 23.20*(rhat/1.01)^(bigT-1)) %>% #tutaj zmienić 23.30na inna kwote i  d=1 na d=(wartosc w poleceniu) czyli zamiast 1.01 to 1.05
  dplyr::mutate(survSRM = (rhat)^(bigT-1),
  dplyr::select(bigT, survival, survSRM, clv, clvSRM)

model_data
sum(model_data$clv)

### ZADANIE 5 Z PREZENTACJI O MODELU RETENCJI (do dopalenia w R)
Oblicz i podaj liczbę klientów, którzy odeszli, stopę retencji (w %) oraz oczekiwany czas i medianę odejścia. Wyniki zaokrąglij do części całkowitych. Zinterpretuj oczekiwany czas i medianę odejścia. Oznaczenia zmiennych:
bigT – zmienna czasowa (miesiące)
cancel – status klienta (1-odejście, 0-ocenzurowanie)
count – liczebność klientów


In [ ]:
# creating dataset #tutaj bedzie excel na 99%
service1yr <- data_frame(bigT   = c(2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 1, 3, 4, 5,  6,  7,  8,  9, 10, 11,  12), 
                         cancel = c(1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1, 0, 0, 0, 0,  0,  0,  0,  0,  0,  0,   0), 
                         count  = c(4, 16, 20, 37, 28, 61, 24, 19, 13, 10, 13, 3, 2, 1, 7, 33, 49, 63, 30, 16, 34, 188))


# descriptive statistics
summary(service1yr)

service1yr %>% summarise(weighted_sum_cancel = sum(cancel*count),
                         weighted_sum_bigT = sum(bigT*count))

# estimating retention rate

# cancels - Number Cancels
# flips - Opportunities to Cancel
# Rhat - Retention Rate (r)
# ET - E(T)
# median - Median(T)

service1yr %>% 
  dplyr::summarise(cancels = sum(cancel*count),
                         flips = sum(bigT*count),
                         Rhat = 1 - sum(cancel*count)/sum(bigT*count)) %>%
  dplyr::mutate(ET = 1/(1-Rhat),
                median = 1+log(.5)/log(Rhat)) # z tego outputu odczytac wyniki i to koniec zadania
        
        
# output dla danych z zajęć: 
#cancels-245 - liczba klinetow co odeszli, flips-5828 - szanse na odejscie klinetow, Rhat(stopa retencji)- 0.958 (prawie 96% szans)
# ET(wartosc oczekiwana odejscia) - 23.8 (prawie 24), mediana - 17.1 (ona mowi 17)